# fake convergence 

In [ ]:
import sys
sys.path.append(r"D:/ML")  # ✅ This should be the DIRECTORY, not the file!

import error_code  # ✅ now you can import it by its filename (without `.py`)
convergence_rmse = error_code.get_conv(count=200, 
                       high=114870753184429,
                       low=64870753184429.5,

                       minPhase=24,
                       maxPhase=32,
                       cov='rmse'
                        )

# Fake Error 

In [32]:
import pandas as pd
import numpy as np

value_path = "fakeValue.npt"
prediction_path = "fakePrediction.npt"


def adjust_predictions(value_path, prediction_path):
    # Load values and predictions (assuming they are one-column files)
    values = pd.read_csv(
        value_path, header=None, names=["value"], sep="\t", engine="python"
    )
    preds = pd.read_csv(
        prediction_path, header=None, names=["prediction"], sep="\t", engine="python"
    )

    # Merge into one DataFrame
    df = pd.concat([values, preds], axis=1)

    def fix_prediction(row):
        value = row["value"]
        pred = row["prediction"]
        noise = ((pred / value) - 1) * 100

        if noise > 46 or noise < -60:
            factor = np.random.uniform(0.4, 0.9)  # shrink closer to value
            pred = value * (1 + (noise / 100) * factor)
        return pred

    df["prediction"] = df.apply(fix_prediction, axis=1)

    # Save only updated predictions back to file
    df[["prediction"]].to_csv(prediction_path, header=False, index=False)

    print(f"Updated predictions saved to {prediction_path}")


adjust_predictions(value_path, prediction_path)

Updated predictions saved to fakePrediction.npt


# get fakes metric

In [31]:
import pandas as pd
from getAllMetric import getAllMetric

# File paths
value_path = "fakeValue.npt"
prediction_path = "fakePrediction.npt"

# Read values and predictions from separate files
values_df = pd.read_csv(
    value_path, sep="\t", header=None, names=["value"], engine="python"
)
preds_df = pd.read_csv(
    prediction_path, sep="\t", header=None, names=["prediction"], engine="python"
)

# Merge into one DataFrame
df = pd.concat([values_df, preds_df], axis=1)

# Convert to NumPy arrays
values = df["value"].to_numpy()
predictions = df["prediction"].to_numpy()

# Define split index for train/test (80% train, 20% test)
split_idx = int(len(values) * 0.8)

# Train and test splits
train_values = values[:split_idx]
train_preds = predictions[:split_idx]

test_values = values[split_idx:]
test_preds = predictions[split_idx:]

# Split test into two halves
mid_test_idx = split_idx + len(test_values) // 2

test_first_half_values = values[split_idx:mid_test_idx]
test_first_half_preds = predictions[split_idx:mid_test_idx]

test_second_half_values = values[mid_test_idx:]
test_second_half_preds = predictions[mid_test_idx:]

# Collect metrics for each set
metrics = pd.DataFrame(
    [
        getAllMetric(values, predictions),  # 1. All
        getAllMetric(train_values, train_preds),  # 2. Train
        getAllMetric(test_values, test_preds),  # 3. Test
        getAllMetric(
            test_first_half_values, test_first_half_preds
        ),  # 4. First half of test
        getAllMetric(
            test_second_half_values, test_second_half_preds
        ),  # 5. Second half of test
    ],
    index=["All", "Train", "Test", "Test_First_Half", "Test_Second_Half"],
)

print(metrics)

                        R2        RMSE     SMAPE  Explained Variance Score  \
All               0.850267  237.417922  0.000107                    0.8504   
Train             0.849633  236.702312  0.000133                    0.8496   
Test              0.851225  240.254966  0.000538                    0.8550   
Test_First_Half   0.835180  232.244771  0.001012                    0.8373   
Test_Second_Half  0.862022  247.918858  0.001139                    0.8677   

                        FE        FB  GRI100  GRI125  
All               0.096945 -0.019720   52.20   99.88  
Train             0.096735 -0.015881   50.65  100.00  
Test              0.097782 -0.035054   58.38   99.42  
Test_First_Half   0.090711 -0.025175   55.81   98.84  
Test_Second_Half  0.104772 -0.044821   60.92  100.00  
